IMPORTING ALL NECESSARY LIBRARIES


In [1]:
#Importing the necessary Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from  xgboost  import  XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet,Ridge,Lasso,LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import warnings



In [2]:
index_names = ['unit_number', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']

sensor_names= ["(Fan inlet temperature) (◦R)",
"(LPC outlet temperature) (◦R)",
"(HPC outlet temperature) (◦R)",
"(LPT outlet temperature) (◦R)",
"(Fan inlet Pressure) (psia)",
"(bypass-duct pressure) (psia)",
"(HPC outlet pressure) (psia)",
"(Physical fan speed) (rpm)",
"(Physical core speed) (rpm)",
"(Engine pressure ratio(P50/P2)",
"(HPC outlet Static pressure) (psia)",
"(Ratio of fuel flow to Ps30) (pps/psia)",
"(Corrected fan speed) (rpm)",
"(Corrected core speed) (rpm)",
"(Bypass Ratio) ",
"(Burner fuel-air ratio)",
"(Bleed Enthalpy)",
"(Required fan speed)",
"(Required fan conversion speed)",
"(High-pressure turbines Cool air flow)",
"(Low-pressure turbines Cool air flow)" ]
col_names = index_names + setting_names + sensor_names

In [3]:
dftrain = pd.read_csv(r'C:\projects\mlproject2\notebook\data\Turbofan.csv',sep='\s+',header=None,index_col=False,names=col_names)
dfvalid = pd.read_csv(r'C:\projects\mlproject2\notebook\data\Test.csv',sep='\s+',header=None,index_col=False,names=col_names)
y_valid = pd.read_csv(r'C:\projects\mlproject2\notebook\data\RULTest.csv',sep='\s+',header=None,index_col=False,names=['RUL'])


In [4]:
train=dftrain.copy()
valid=dfvalid.copy()

In [5]:
train['RUL'] = train.groupby(['unit_number'])['time_cycles'].transform(max)-train['time_cycles']
y=train['RUL']

In [6]:
X=train.drop(['RUL'],axis=1)
X

,unit_number,time_cycles,setting_1,setting_2,setting_3,(Fan inlet temperature) (◦R),(LPC outlet temperature) (◦R),(HPC outlet temperature) (◦R),(LPT outlet temperature) (◦R),(Fan inlet Pressure) (psia),...,(Ratio of fuel flow to Ps30) (pps/psia),(Corrected fan speed) (rpm),(Corrected core speed) (rpm),(Bypass Ratio),(Burner fuel-air ratio),(Bleed Enthalpy),(Required fan speed),(Required fan conversion speed),(High-pressure turbines Cool air flow),(Low-pressure turbines Cool air flow)
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640


In [7]:
numeric_features=X.select_dtypes(exclude="object").columns



In [11]:
from sklearn.preprocessing import  MinMaxScaler
from sklearn.compose import ColumnTransformer
numeric_transformer = MinMaxScaler()
#oh_transformer = OneHotEncoder()

preprocessor= ColumnTransformer(
    [
        ("MinMaxScaler",numeric_transformer,numeric_features),
        
    ]
)

In [12]:
X=preprocessor.fit_transform(X)

In [13]:
X

array([[0.        , 0.        , 0.45977011, ..., 0.        , 0.71317829,
        0.7246617 ],
       [0.        , 0.00277008, 0.6091954 , ..., 0.        , 0.66666667,
        0.73101353],
       [0.        , 0.00554017, 0.25287356, ..., 0.        , 0.62790698,
        0.62137531],
       ...,
       [1.        , 0.54570637, 0.52298851, ..., 0.        , 0.23255814,
        0.05399061],
       [1.        , 0.54847645, 0.43678161, ..., 0.        , 0.11627907,
        0.23446562],
       [1.        , 0.55124654, 0.31609195, ..., 0.        , 0.17829457,
        0.21817178]])

In [14]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.3,random_state=42)
X_train.shape,X_test.shape

((14441, 26), (6190, 26))

In [ ]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true, predicted)
    mse=np.sqrt(mean_squared_error(true,predicted))
    r2_score= r2_score(true,predicted)
    return mae, mse ,r2_score

DEFINING FEATURES